# Import libraries

In [1]:
import os
import sys
import tweepy
import requests
from dotenv import load_dotenv

sys.path.append('../../')
import config

load_dotenv(os.path.join(config.PROJ_PATH, '.env')) 

True

# Set twitter access

In [2]:
consumer_key = os.environ["API_KEY"]
consumer_secret = os.environ["API_KEY_SECRET"]
access_token = os.environ["ACCESS_TOKEN"]
access_token_secret = os.environ["ACCESS_TOKEN_SECRET"]

auth = tweepy.OAuth1UserHandler(
  consumer_key, 
  consumer_secret, 
  access_token, 
  access_token_secret
)

api = tweepy.API(auth)

# Create functions to read and save texts and images

In [3]:
def save_tweet_text(tweet:object, text_type='') -> None:
    tweet_timestamp = str(tweet.created_at.strftime("%Y-%m-%d %H-%M-%S"))
    tweet_id = str(tweet.id)
    txt_file_name = os.path.join(config.DATA_PATH_RAW_TEXTS, '{}_{}.txt'.format(tweet_timestamp, tweet_id))
    
    with open(txt_file_name, 'w', encoding="utf-8") as txt_file:
        try:
            if text_type == 'retweet':
                txt_file.write(tweet.retweeted_status.full_text)
            else:
                txt_file.write(tweet.full_text)
            # txt_file.write(tweet.retweeted_status.full_text)
        except AttributeError:
            print('Unable to read text from tweet {}'.format(tweet.id))
            print("=====")

    return None

def DownloadFile(url:str, path_to_save:str) -> None:
    response = requests.get(url)

    if response.status_code == 200:
        with open(path_to_save, 'wb') as f:
                f.write(response.content)
    return None


def save_tweet_image(tweet:object) -> None:
    tweet_timestamp = str(tweet.created_at.strftime("%Y-%m-%d %H-%M-%S"))
    tweet_id = str(tweet.id)

    try:
        for media in tweet.entities.get("media",[{}]):
            #checks if there is any media-entity
            if media.get("type",None) == "photo":
                tweet_media_id = str(media['id'])
                filename = os.path.join(config.DATA_PATH_RAW_IMAGES, '{}_{}_{}.png'.format(tweet_timestamp, tweet_id, tweet_media_id))
                DownloadFile(media["media_url"], filename)
    except AttributeError:
        print('Unable to read medias from tweet {}'.format(tweet_id))
        print("=====")

# Pull tweets

In [4]:
phrases_related_to_vaw = ['que ficar em casa', 'mulher obediente', 'é divorciada', 'Ninguém vai acreditar em você', 
    'Não presta nem pra cozinha', 'Mal sabe lavar uma roupa direito', 'burra', 'interesseira']

In [5]:
tweets_pages = []
for status in tweepy.Cursor(api.search_tweets,
                            'noticia', 
                            tweet_mode="extended", 
                            lang='pt', 
                            count=1).pages(1):
    tweets_pages.append(status)

# Read tweets
for page in tweets_pages:
    for tweet in page:
        # save_tweet_text(tweet)
        # print(tweet.full_text)

        # save_tweet_image(tweet)

        if tweet.coordinates is not None:
            print(tweet.coordinates)
            print(tweet.geo)
            print(tweet.contributors)
            break

In [8]:
# Usefull tweet attributes [author(id, name, screen_name, location), user(id, name, screen_name, location), 
#                           geo, place, coordinates]
vars(tweet.author).keys()

dict_keys(['_api', '_json', 'id', 'id_str', 'name', 'screen_name', 'location', 'description', 'url', 'entities', 'protected', 'followers_count', 'friends_count', 'listed_count', 'created_at', 'favourites_count', 'utc_offset', 'time_zone', 'geo_enabled', 'verified', 'statuses_count', 'lang', 'contributors_enabled', 'is_translator', 'is_translation_enabled', 'profile_background_color', 'profile_background_image_url', 'profile_background_image_url_https', 'profile_background_tile', 'profile_image_url', 'profile_image_url_https', 'profile_banner_url', 'profile_link_color', 'profile_sidebar_border_color', 'profile_sidebar_fill_color', 'profile_text_color', 'profile_use_background_image', 'has_extended_profile', 'default_profile', 'default_profile_image', 'following', 'follow_request_sent', 'notifications', 'translator_type', 'withheld_in_countries'])

In [17]:
tweet.author.utc_offset